In [0]:
from pyspark.sql.functions import col, lit
from pyspark.sql.functions import input_file_name, substring_index,col
parquet_file_path1 = "dbfs:/Silver/2_wheelers/202403/29/*.parquet"
parquet_file_path2 = "dbfs:/Silver/4_wheelers/202403/29/*.parquet"

from pyspark.sql.functions import col

df1 = spark.read.parquet(parquet_file_path1).select(col('dealerlocation').alias('City'), col('region').alias('Region'), lit('Unknown').alias('Country')).distinct()


display(df1)

df2= spark.read.parquet(parquet_file_path2).select(col('country').alias('Country'), col('city').alias('City'), lit('Unknown').alias('Region')).distinct()

display(df2)



City,Region,Country
Bangalore,South,Unknown
Ahmedabad,West,Unknown
Hyderabad,South,Unknown
NoCity,West,Unknown
Kolkata,East,Unknown
Jaipur,North,Unknown
Chennai,South,Unknown
#####,West,Unknown
NULL,West,Unknown
Mumbai,West,Unknown


Country,City,Region
India,Hyderabad,Unknown
India,Ahmedabad,Unknown
India,Bangalore,Unknown
India,Jaipur,Unknown
India,Mumbai,Unknown
India,Kolkata,Unknown
India,Lucknow,Unknown
India,Delhi,Unknown
India,Chennai,Unknown
India,Pune,Unknown


In [0]:
#nnnn
from pyspark.sql.functions import col, input_file_name, lit, when, split, regexp_replace
import pyspark.sql.functions as F

stg_df_1 = spark.read.format('parquet').option('Header', 'True').option('inferSchema', 'True').load(parquet_file_path1) \
    .withColumn('inputfilename', input_file_name()).withColumn('inputfilename', split(F.col('inputfilename'), '/').getItem(2)).withColumn('WheelerKey', when(F.col('inputfilename') == '2-WheelSales', lit('F')).otherwise(lit('T'))).select(col('dealerlocation').alias('City'), col('region').alias('Region'), lit('Unknown').alias('Country'), 'wheelerkey')

stg_df_2 =  spark.read.format('parquet').option('Header', 'True').option('inferSchema', 'True').load(parquet_file_path2) \
    .withColumn('inputfilename', input_file_name()).withColumn('inputfilename', split(col('inputfilename'), '/').getItem(2)).withColumn('WheelerKey', when(F.col('inputfilename') == '4-WheelSales', lit('T')).otherwise(lit('F'))).select(col('city').alias('City'), col('country').alias('Country'), lit('Unknown').alias('Region'), 'wheelerkey')

combined_stg = stg_df_1.unionByName(stg_df_2)
print("combined_count :", combined_stg.count())
# pattern = " "
# combined_stg = combined_stg.withColumn("City", regexp_replace(col("City"), pattern, "")).dropDuplicates(['City','Region','Country'])
combined_stg=combined_stg.dropDuplicates(['City','Region','Country'])
print("combined_distinct :" ,combined_stg.count())
combined_stg.show()

dim_df = spark.sql("""SELECT * FROM  dev.dimlocation""")
print('dim_df show...')
dim_df.show()
print("check for new records...")

new_records = combined_stg.join(dim_df, 'City', 'left_anti')
new_records.show()

updated_records = combined_stg.join(dim_df, combined_stg['City'] == dim_df['City'], 'inner').select(dim_df['*'])

print("matching from src to target ")
updated_records.show()

combined_records = new_records.unionByName(updated_records.drop('pkLocationId'))


print('combined....')
combined_records = combined_records.dropDuplicates(['City','Region','Country'])
print('combined_records df count:', combined_records.count())
combined_records.show()

combined_records.write.mode('overwrite').saveAsTable("dev.dimlocation")

print("success") 

combined_count : 116786
combined_distinct : 26
+-----------+-------+-------+----------+
|       City| Region|Country|wheelerkey|
+-----------+-------+-------+----------+
|      #####|   West|Unknown|         T|
|     AFMDSS|  #####|Unknown|         T|
|  Ahmedabad|Unknown|  India|         F|
|  Ahmedabad|   West|Unknown|         T|
|  Bangalore|  South|Unknown|         T|
|  Bangalore|Unknown|  India|         F|
|Bhubaneswar|   East|Unknown|         T|
| Chandigarh|  North|Unknown|         T|
|    Chennai|  South|Unknown|         T|
|    Chennai|Unknown|  India|         F|
|      Delhi|  North|Unknown|         T|
|      Delhi|Unknown|  India|         F|
|   Guwahati|   East|Unknown|         T|
|  Hyderabad|  South|Unknown|         T|
|  Hyderabad|Unknown|  India|         F|
|     Jaipur|  North|Unknown|         T|
|     Jaipur|Unknown|  India|         F|
|    Kolkata|   East|Unknown|         T|
|    Kolkata|Unknown|  India|         F|
|    Lucknow|Unknown|  India|         F|
+---------

In [0]:
%sql
select * from dev.dimlocation

pkLocationId,Country,Region,City,WheelerKey
1,Unknown,West,#####,T
2,Unknown,#####,AFMDSS,T
3,India,Unknown,Ahmedabad,F
4,Unknown,West,Ahmedabad,T
5,Unknown,South,Bangalore,T
6,India,Unknown,Bangalore,F
7,Unknown,East,Bhubaneswar,T
8,Unknown,North,Chandigarh,T
9,Unknown,South,Chennai,T
10,India,Unknown,Chennai,F


In [0]:
# from pyspark.sql.functions import col, lit

# # For DataFrame df1
# df1_unique = df1.select('City', 'Region', 'Country').distinct().count()

# # For DataFrame df2
# df2_unique = df2.select('City', 'Country', 'Region').distinct().count()

# # Display unique records
# print(df1_unique)


# print(df2_unique)



16
10


In [0]:
# from pyspark.sql.functions import *
# from pyspark.sql.types import *
# from datetime import datetime 

# def Location_scd1(path1, path2):

#     stg_df_2 = spark.read.format('parquet') \
#                     .option('Header', 'True') \
#                     .load(path1) \
#                     .withColumn('inputfilename', input_file_name()) \
#                     .withColumn('inputfilename',  concat_ws("/", expr("slice(split(inputfilename, '/'), 1, 3)"))) \
#                     .withColumn('wheelerkey', when(col('inputfilename') == 'dbfs:/Silver/2_wheelers', lit('T')).otherwise(lit('F'))) \
#                     .select(col('dealerlocation').alias('City'), col('region').alias('Region'), lit('Unknown').alias('Country'), 'wheelerkey').distinct()


#     stg_df_4 = spark.read.format('parquet') \
#                     .option('Header', 'True') \
#                     .load(path2) \
#                     .withColumn('inputfilename', input_file_name()) \
#                     .withColumn('inputfilename',  concat_ws("/", expr("slice(split(inputfilename, '/'), 1, 3)"))) \
#                     .withColumn('wheelerkey', when(col('inputfilename') == 'dbfs:/Silver/4_wheelers', lit('F')).otherwise(lit('T'))) \
#                     .select(col('city').alias('City'), col('country').alias('Country'), lit('Unknown').alias('Region'), 'wheelerkey').distinct()


#     # # Load Delta table
#     # delta_table = DeltaTable.forPath(spark, "dbfs:/delta_tables/Dim1")

#     combined_stg = stg_df_2.unionByName(stg_df_4)

#     combined_stg.show()

#     print("read the table")

#     dim_df = spark.sql("select * from dev.dimlocation ")
#     print(dim_df.printSchema())
#     # Cast the "City" column to StringType in dim_df
#     dim_df = dim_df.withColumn("City", dim_df["City"].cast(StringType()))

#     dim_df.show()
#     print("check for new and updated records...")
#     new_records = combined_stg.join(dim_df, 'City', 'left_anti')
#     new_records.show()

#     print("check for matched records")

#     matched_records = combined_stg.join(dim_df,  combined_stg['city']==dim_df['City'], 'inner').select(dim_df['*'])

#     matched_records.show()

#     print("final_df")

#     final_df = matched_records.drop('pkLocationId').unionByName(new_records)

#     final_df.show()


#     final_df.write.mode('overwrite').saveAsTable('dev.dimlocation')

#     # final_df1 = final_df.fillna('India', subset=['Country'])
#     # final_df1.show()

#     print('success')



    

In [0]:
Location_scd1(parquet_file_path1, parquet_file_path2)

+-----------+-------+-------+----------+
|       City| Region|Country|wheelerkey|
+-----------+-------+-------+----------+
|     NoCity|   West|Unknown|         T|
|    Kolkata|   East|Unknown|         T|
|  Ahmedabad|   West|Unknown|         T|
|       NULL|   West|Unknown|         T|
|    Chennai|  South|Unknown|         T|
|   Guwahati|   East|Unknown|         T|
| Chandigarh|  North|Unknown|         T|
|      #####|   West|Unknown|         T|
|  Hyderabad|  South|Unknown|         T|
|     Mumbai|   West|Unknown|         T|
|  Bangalore|  South|Unknown|         T|
|Bhubaneswar|   East|Unknown|         T|
|     Jaipur|  North|Unknown|         T|
|       Pune|   West|Unknown|         T|
|     AFMDSS|  #####|Unknown|         T|
|      Delhi|  North|Unknown|         T|
|  Ahmedabad|Unknown|  India|         F|
|      Delhi|Unknown|  India|         F|
|     Jaipur|Unknown|  India|         F|
|    Lucknow|Unknown|  India|         F|
+-----------+-------+-------+----------+
only showing top

In [0]:
%sql
select * from dev.dimlocation

pkLocationId,Country,Region,City,WheelerKey
1,Unknown,West,NoCity,T
3,Unknown,East,Kolkata,T
5,Unknown,West,Ahmedabad,T
7,Unknown,West,NULL,T
9,Unknown,South,Chennai,T
11,Unknown,East,Guwahati,T
13,Unknown,North,Chandigarh,T
15,Unknown,West,#####,T
17,Unknown,South,Hyderabad,T
19,Unknown,West,Mumbai,T
